# 6章

In [3]:
from dotenv import load_dotenv
from pathlib import Path

env_path = Path(".") / ".env"
load_dotenv(dotenv_path=env_path)  # 環境変数を読み込む

True

In [4]:
# ドキュメントは別リポジトリで管理されている

# from langchain_community.document_loaders import GitLoader


# def file_filter(file_path: str) -> bool:
#     return file_path.endswith(".mdx")A


# loader = GitLoader(
#     clone_url="https://github.com/langchain-ai/langchain",
#     repo_path="./langchain",
#     branch="master",
#     file_filter=file_filter,
# )

# documents = loader.load()
# print(len(documents))


### データベース化対象

In [5]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/docs",
    repo_path="./docs",
    branch="main",
    file_filter=file_filter,
)

documents = loader.load()
print("Loaded documents:", len(documents))


Loaded documents: 1979


### ベクトル化

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import random
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# ----------------------
# 2. サンプリングで件数削減
# ----------------------
sample_fraction = 0.25
sampled_documents = random.sample(documents, k=int(len(documents) * sample_fraction))
print("Sampled documents:", len(sampled_documents))

# ----------------------
# 3. チャンク化
# ----------------------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,  # 1チャンクあたり最大4000文字
    chunk_overlap=100,  # 200文字重複で文脈をつなぐ
)

split_docs = text_splitter.split_documents(sampled_documents)
print("Split into chunks:", len(split_docs))

Sampled documents: 494
Split into chunks: 1613


In [7]:
# ----------------------
# 4. Embeddings 作成 & Chroma登録
# ----------------------
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma(embedding_function=embeddings, persist_directory="./chroma_db")

In [8]:
from tqdm.notebook import tqdm  # ←Notebook用

batch_size = 100
for i in tqdm(range(0, len(split_docs), batch_size)):
    batch = split_docs[i : i + batch_size]
    db.add_documents(batch)

  0%|          | 0/17 [00:00<?, ?it/s]

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("LangChainの概要を教えて")


'LangChainは、主に自然言語処理やAIアプリケーションの開発を支援するためのSDK（ソフトウェア開発キット）です。特に、JavaScript向けのモジュールが提供されており、開発者はこれを利用して言語モデルやデータ処理の機能を簡単に統合できます。また、LangChain Academyというリソースもあり、ここではLangChainの使い方や関連技術について学ぶことができます。詳細な情報は、[LangChain SDKの公式ドキュメント](https://reference.langchain.com/javascript/modules/langchain.html)や[LangChain Academy](https://academy.langchain.com/)で確認できます。'

## 6.3

### HyDE

In [10]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()


In [11]:
hyde_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hypothetical_chain | retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

hyde_rag_chain.invoke("LangChainの概要を教えて")


'LangChainは、言語モデル（LLM）を使用してアプリケーションを構築するためのフレームワークです。主に以下の特徴があります：\n\n1. **簡単なスタート**: LangChainは、LLMを使ったアプリケーションの構築を容易にすることを目的としています。開発者がすぐに始められるように設計されています。\n\n2. **柔軟性と生産性**: LangChainは、さまざまなモデルプロバイダーやデータソースと統合できる柔軟性を持ち、プロダクション環境でも使用できるように設計されています。\n\n3. **コンポーネントのチェーン化**: LangChainは、プロンプトテンプレート、モデル、出力パーサーなどの基本的なコンポーネントを使用して、複雑なフローを構築することができます。これにより、開発者は異なるコンポーネントを組み合わせて、より高度なアプリケーションを作成できます。\n\n4. **エージェント的なアプリケーション**: LangChainは、将来的なアプリケーションがよりエージェント的になると考えており、LLMを使ってデータや計算と相互作用する複雑なフローをオーケストレーションすることを目指しています。\n\n5. **トレーシングとデプロイ**: LangSmithを使用することで、アプリケーションのトレーシングやデバッグが可能になり、LangServeを使ってアプリケーションをREST APIとしてデプロイすることもできます。\n\nLangChainは、開発者が最新のモデルを簡単に利用できるようにし、アプリケーションの構築を効率化することを目指しています。'

### 複数の検索クエリの生成


In [12]:
import pprint
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")


pprint.pprint(QueryGenerationOutput.model_json_schema())

{'properties': {'queries': {'description': '検索クエリのリスト',
                            'items': {'type': 'string'},
                            'title': 'Queries',
                            'type': 'array'}},
 'required': ['queries'],
 'title': 'QueryGenerationOutput',
 'type': 'object'}


In [13]:
query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

pprint.pprint(query_generation_chain)

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='質問に対してベクターデータベースから関連文書を検索するために、\n3つの異なる検索クエリを生成してください。\n距離ベースの類似性検索の限界を克服するために、\nユーザーの質問に対して複数の視点を提供することが目標です。\n\n質問: {question}\n'), additional_kwargs={})])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x70820ac21940>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x70820ac23380>, root_client=<openai.OpenAI object at 0x70820ade6850>, root_async_client=<openai.AsyncOpenAI object at 0x70820ac34190>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True), kwargs={'response_format': <class '__main__.QueryGenerationOutput'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': 

In [14]:
multi_query_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": query_generation_chain | retriever.map(),
    }
    | prompt
    | model
    | StrOutputParser()
)

out = multi_query_rag_chain.invoke("LangChainの概要を教えて")
print(out)

LangChainは、言語モデル（LLM）を活用したアプリケーションを構築するためのフレームワークです。主な特徴は以下の通りです：

1. **モジュール性**: LangChainは、さまざまなモジュールを提供しており、これらを単独で使用したり、複雑なユースケースのために組み合わせたりすることができます。

2. **コンポーネントのチェーン**: LangChainは、コンポーネントをシームレスに連結するためのLangChain Expression Language（LCEL）を使用しており、これにより複雑なフローを簡単に構築できます。

3. **基本的なコンポーネント**: LangChainのアプリケーションは、主に以下の3つのコンポーネントで構成されます：
   - **LLM/チャットモデル**: 言語モデルのコアエンジン。
   - **プロンプトテンプレート**: 言語モデルに指示を与えるためのもの。
   - **出力パーサー**: 言語モデルからの生の応答をより扱いやすい形式に変換します。

4. **開発とデプロイ**: LangChainは、アプリケーションの開発、テスト、デプロイをサポートするためのツール（LangSmithやLangServeなど）を提供しています。LangSmithはアプリケーションのトレースやデバッグを行うためのプラットフォームであり、LangServeはLangChainのチェーンをREST APIとしてデプロイするためのライブラリです。

5. **柔軟性と拡張性**: LangChainは、さまざまなモデルプロバイダーやデータストア、APIとの統合を容易にし、開発者が最新のモデルを簡単に利用できるように設計されています。

LangChainは、言語モデルを利用したアプリケーションの開発を簡素化し、将来的なアプリケーションの可能性を広げることを目指しています。


## 6.4

### RAG Fusion


In [15]:
from langchain_core.documents import Document


def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 各ドキュメントのコンテンツ (文字列) とそのスコアの対応を保持する辞書を準備
    content_score_mapping = {}

    # 検索クエリごとにループ
    for docs in retriever_outputs:
        # 検索結果のドキュメントごとにループ
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 初めて登場したコンテンツの場合はスコアを0で初期化
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (順位 + k)) のスコアを加算
            content_score_mapping[content] += 1 / (rank + k)

    # スコアの大きい順にソート
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]


In [16]:
rag_fusion_chain = (
    {
        "question": RunnablePassthrough(),
        "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
    }
    | prompt
    | model
    | StrOutputParser()
)

rag_fusion_chain.invoke("LangChainの概要を教えて")


'LangChainは、言語モデル（LLM）を使用してアプリケーションを構築するためのフレームワークです。主な目的は、開発者がLLMを簡単に利用できるようにし、柔軟で生産性の高いアプリケーションを作成できるようにすることです。LangChainは、以下のような特徴を持っています。\n\n1. **モジュールの統合**: LangChainは、プロンプトテンプレート、モデル、出力パーサーなどの基本的なコンポーネントを提供し、これらを組み合わせて複雑なフローを構築できます。\n\n2. **エコシステムのサポート**: LangChainは、OpenAIなどのモデルプロバイダーやデータストア、APIとの統合を必要とし、これにより外部データソースと連携したアプリケーションの開発が可能です。\n\n3. **トレーシングとデバッグ**: LangSmithというツールを使用することで、アプリケーションのトレーシングやデバッグが容易になり、複雑なアプリケーションの内部を可視化できます。\n\n4. **デプロイメント**: LangServeを使用することで、LangChainのチェーンをREST APIとしてデプロイすることができます。\n\n5. **進化する技術**: LangChainは、LLMの進化に合わせて常に更新されており、開発者が最新の技術を利用できるようにしています。\n\nLangChainは、開発者がLLMを活用して、よりインテリジェントでエージェント的なアプリケーションを構築するための基盤を提供します。'

### Cohere のリランクモデルの導入


In [17]:
# バージョン合わなかったので略

## 6.5

### LLM によるルーティング


In [18]:
from langchain_community.retrievers import TavilySearchAPIRetriever


langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config({"run_name": "web_retriever"})


In [19]:
from enum import Enum


class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"


class RouteOutput(BaseModel):
    route: Route


route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。

質問: {question}
""")

route_chain = (
    route_prompt | model.with_structured_output(RouteOutput) | (lambda x: x.route)
)

In [20]:
from typing import Any


def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown route: {route}")


route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt
    | model
    | StrOutputParser()
)


In [21]:
route_rag_chain.invoke("LangChainの概要を教えて")


'LangChainは、主に自然言語処理やAIアプリケーションの開発を支援するためのSDK（ソフトウェア開発キット）です。LangChainを使用することで、開発者は言語モデルを活用したアプリケーションを簡単に構築できるようになります。公式のリファレンスや学習リソースは、LangChainの機能や使い方を学ぶために提供されています。具体的には、LangChainのリファレンスは[こちら](https://reference.langchain.com/javascript/modules/langchain.html)で、学習リソースは[こちら](https://academy.langchain.com/)からアクセスできます。'

In [22]:
route_rag_chain.invoke("東京の今日の天気は？")


'東京の今日の天気は、午前中に雨が降る可能性があり、その後は日差しが出るチャンスもありますが、夕方以降は再び雨の可能性があります。外出する際には雨具を持っていくことをおすすめします。また、全体的に曇りがちで、気温は低めになる見込みです。'

### ハイブリッド検索の実装


In [23]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config({"run_name": "chroma_retriever"})

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)


In [ ]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel(
        {
            "chroma_documents": chroma_retriever,
            "bm25_documents": bm25_retriever,
        }
    )
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion  # 最後にソート
)


In [25]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChainの概要を教えて")


'LangChainは、AIアプリケーションを構築するためのフレームワークであり、特に大規模言語モデル（LLM）を活用したアプリケーションやエージェントの開発を支援します。LangChainを使用することで、開発者は複数のAPI呼び出しを組み合わせてユーザーのリクエストに応じた応答を生成することができます。\n\nLangChainは、以下のような機能を提供します：\n\n1. **エージェントの構築**: ユーザーのリクエストに基づいて、複数のツールやAPIを組み合わせて処理を行うエージェントを作成できます。\n2. **ログとトレース**: Portkeyとの統合により、すべてのリクエストをログに記録し、トレースIDを使用してリクエストの可視化を行うことができます。これにより、ユーザーのインタラクションを詳細に分析できます。\n3. **キャッシングとリトライ**: 過去のリクエストをキャッシュして再利用することで、コストを削減し、応答時間を短縮することができます。また、失敗したリクエストを自動的に再処理する機能も提供されています。\n4. **セマンティックな制御**: Pebbloとの統合により、データの読み込みと取得において、セマンティックなトピックやエンティティに基づく制御を行うことができます。\n\nLangChainは、さまざまなAIプロバイダーやモデルに接続できるため、開発者は柔軟にアプリケーションを構築し、運用することが可能です。'